First, we initialize the matroid and obtain the lattice of flats. We relabel the flats for simple presentation.

In [2]:
r, n = 5, 6

R.<q> = PolynomialRing(QQ)
matroid = matroids.Uniform(r, n)
flats = list(matroid.lattice_of_flats())

In [3]:
#plottting the geometric lattice:
#size = 0 for now

labels = {element: idx for idx, element in enumerate(flats)}
matroid.lattice_of_flats().plot(
    element_labels = labels, element_color = "white",
    figsize= 0, cover_color = "grey")

Graphics object consisting of 230 graphics primitives

In [4]:
def invKL(M):
	R = PolynomialRing(QQ, 'q')
	q = R.gen(0)
	if M.rank() == 0:
		return R(1)
	L = M.lattice_of_flats()
	invKLpoly = 0
	for i in range(1,len(L)):
		Mcon = M.contract(L[i])
		Mres = M.delete(M.groundset()-L[i])
		Lres = Mres.lattice_of_flats()
		invKLpoly += (-1)^(Mres.rank())*Lres.kazhdan_lusztig_polynomial()*invKL(Mcon)
	return -invKLpoly

Guess 1:
$$Q_M(t) = Q_{M\setminus e}(t)-tQ_{M/e}(t)+\sum_{F\in S}{\tau(M/{F\cup e})t^{\text{crk}(F)/2}Q_{M\mid F}(t)}.$$

Functions needed for the r.h.s. will be written below:

In [5]:
#the right hand side of the equality

def q_tau(matroid, S, e):
    q_tau = [] 
    for F in S:
        modded_matroid = matroid.contract(F.union(e))
        if modded_matroid.rank() % 2 == 0:
            q_tau.append(0)
        else:
            q_tau.append(invKL(modded_matroid).leading_coefficient())
    return q_tau

def tau(matroid, S, e):
    tau = [] 
    for F in S:
        modded_matroid = matroid.contract(F.union(e))
        if modded_matroid.rank() % 2 == 0:
            tau.append(0)
        else:
            tau.append(modded_matroid.lattice_of_flats().kazhdan_lusztig_polynomial().leading_coefficient())
    return tau

def the_set_S(flats, e):
    return set(F for F in flats if e.isdisjoint(F) and F.union(e) in flats)

def delform_rhs(matroid, tau_func, inv_kl):
    e = matroid.flats(1)[0]
    flats = list(matroid.lattice_of_flats())

    deletion = matroid.delete(e).simplify()
    contraction = matroid.contract(e).simplify()

    set_S = list(the_set_S(flats, e))

    # Compute bv_tau_function and zip with set_S
    tau_values = tau_func(matroid, set_S, e)

    # Compute the summation over S
    sum_S = sum(
        x * q ** ((matroid.rank() - matroid.rank(y))//2) * inv_kl(matroid.delete(matroid.groundset() - y).simplify())
        for x, y in zip(tau_values, set_S)
    )
    # Compute the RHS
    return inv_kl(deletion) - q * inv_kl(contraction) + sum_S

In [6]:
# Create a list of matroids
matroids_list = [matroids.Uniform(r, n) for r, n in [(2, 3), (2, 4), (2,5), (3, 4), (4, 5), (5,6), (3, 5), (4, 6)]] + \
    [matroids.Wheel(i) for i in range(3, 6)] + \
    [matroids.Whirl(i) for i in range(3, 5)]

# Create a dataframe to store the results
import pandas as pd

data = {
    'Matroid': matroids_list,
    'Kazhdan-Lusztig': [m.lattice_of_flats().kazhdan_lusztig_polynomial() for m in matroids_list],
    'invKL': [invKL(m) for m in matroids_list],
    'delform_rhs_qtau': [delform_rhs(m, q_tau, invKL) for m in matroids_list],
    'delform_rhs_tau': [delform_rhs(m, tau, invKL) for m in matroids_list],
    #'difference': [x-y for x, y in zip(data['invKL'], data['delform_rhs_tau'])]
}

df = pd.DataFrame(data)
display(df)

KeyboardInterrupt: 

In [ ]:
def parallel_connection(M1, M2, common_element):
    """
    Compute the parallel connection of two matroids M1 and M2 over a common element.
    
    Parameters:
    M1, M2: Matroid objects.
    common_element: The element over which the parallel connection is formed.
    
    Returns:
    Matroid object representing the parallel connection of M1 and M2.
    """
    # Ensure the common element is in both ground sets
    assert common_element in M1.groundset(), "Common element must be in the ground set of M1"
    assert common_element in M2.groundset(), "Common element must be in the ground set of M2"
    
    # Compute the ground set of the parallel connection
    ground_set = (M1.groundset() | M2.groundset()) - {common_element}
    
    # Compute the circuits of the parallel connection
    circuits = []
    for C1 in M1.circuits():
        if common_element in C1:
            for C2 in M2.circuits():
                if common_element in C2:
                    new_circuit = (C1 | C2) - {common_element}
                    if new_circuit not in circuits:
                        circuits.append(new_circuit)
        else:
            circuits.append(C1)
    
    for C2 in M2.circuits():
        if common_element not in C2:
            circuits.append(C2)
    
    # Create the parallel connection matroid
    parallel_conn_matroid = Matroid(groundset=ground_set, circuits=circuits)
    
    return parallel_conn_matroid

# Example usage
M1 = Matroid(groundset=[0, 1, 2], circuits=[[0, 1], [1, 2]])
M2 = Matroid(groundset=[2, 3, 4], circuits=[[2, 3], [3, 4]])

common_element = 2
parallel_conn = parallel_connection(M1, M2, common_element)
print(parallel_conn)